# Introdução

## Importação das bibliotecas necessarias

In [1]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys

## Funções auxiliares

In [2]:
def return_sql_query(query_string):
    ret = {
        "token": {
            "token": token
        },
        "sql": {
            "sql": {
                "query":query_string
            }
        }
    }
    return ret

def json_to_df(resp):
    columns = [col['name'] for col in resp.json()['columns']]
    values = resp.json()['rows']  
    return pd.DataFrame(values, columns=columns)

def rows_and_columns_to_df(columns, rows):
    columns_array = [col['name'] for col in columns]
    return pd.DataFrame(rows, columns=columns_array)

def fetch_all_data_from_response(res, initial_records_rows):
    while 'cursor' in res.json():
        res_next = {
            "token": {
                "token": token
            },
            "sql": {
                "sql": {
                    "cursor": res.json()['cursor']
                }
            }
        }
        res = requests.post(os.path.join(url_base,'sql_query'), json=res_next)
        initial_records_rows+=res.json()['rows']
        sys.stdout.write('.')
        sys.stdout.flush()
    print('(y)')
    return initial_records_rows

## Dados de acesso a api

In [3]:
from IPython.display import clear_output
import getpass

token = getpass.getpass('Digite o seu token para acesso a API:')

clear_output()

In [4]:
url_base = "https://bigdata-api.fiocruz.br"

## Bases de dados disponiveis na API atualizadas

In [5]:
params = {
          "token": token
        }
resp = requests.post(url_base+ '/show_tables', json=params)
print(resp.json())

{'databases': ['datasus-sim', 'datasus-sinasc', 'datasus-sih']}


## Busca dos campos existentes na base SIM

In [6]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sim"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.11 µs
['ACIDTRAB', 'ALTCAUSA', 'ASSISTMED', 'ATESTADO', 'ATESTANTE', 'CAUSABAS', 'CAUSABAS_O', 'CAUSAMAT', 'CB_PRE', 'CIRCOBITO', 'CIRURGIA', 'CODBAIOCOR', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODIFICADO', 'CODINST', 'CODMUNCART', 'CODMUNNATU', 'CODMUNOCOR', 'CODMUNRES', 'COMUNSVOIM', 'CONTADOR', 'CRM', 'DIFDATA', 'DTATESTADO', 'DTCADASTRO', 'DTCADINF', 'DTCADINV', 'DTCONCASO', 'DTCONINV', 'DTINVESTIG', 'DTNASC', 'DTOBITO', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'ESC', 'ESC2010', 'ESCFALAGR1', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTABDESCR', 'ESTCIV', 'EXAME', 'EXPDIFDATA', 'FONTE', 'FONTEINV', 'FONTES', 'FONTESINF', 'GESTACAO', 'GRAVIDEZ', 'HORAOBITO', 'IDADE', 'IDADEMAE', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'LOCOCOR', 'MORTEPARTO', 'NATURAL', 'NECROPSIA', 'NUDIASINF', 'NUDIASOBCO', 'NUDIASOBIN', 'NUMERODN', 'NUMERODO', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'NUMSUS', 'OBITOGRAV', 'OBITOPARTO', 

## Busca dos campos existentes na base SINASC

In [7]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sinasc"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.87 µs
['APGAR1', 'APGAR5', 'CODANOMAL', 'CODBAINASC', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODINST', 'CODMUNCART', 'CODMUNNASC', 'CODMUNNATU', 'CODMUNRES', 'CODOCUPMAE', 'CODPAISRES', 'CODUFNATU', 'CONSPRENAT', 'CONSULTAS', 'CONTADOR', 'DIFDATA', 'DTCADASTRO', 'DTDECLARAC', 'DTNASC', 'DTNASCMAE', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'DTULTMENST', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTCIVMAE', 'GESTACAO', 'GRAVIDEZ', 'HORANASC', 'IDADEMAE', 'IDADEPAI', 'IDANOMAL', 'KOTELCHUCK', 'LOCNASC', 'MESPRENAT', 'NATURALMAE', 'NOVO', 'NUMERODN', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'ORIGEM', 'PARIDADE', 'PARTO', 'PESO', 'PREFIXODN', 'QTDFILMORT', 'QTDFILVIVO', 'QTDGESTANT', 'QTDPARTCES', 'QTDPARTNOR', 'RACACOR', 'RACACORMAE', 'RACACORN', 'RACACOR_RN', 'SEMAGESTAC', 'SERIESCMAE', 'SEXO', 'STCESPARTO', 'STDNEPIDEM', 'STDNNOVA', 'STTRABPART', 'TPAPRESENT', 'TPDOCRESP', 'TPFUNCRESP', 'TPMETESTIM', 'TPNASCASSI', 'TPROB

# Taxa de fecundidade específica em mulheres de 20 a 34 anos (SINASC + Tabnet)

A ideia inicial era explorar os dados de todo o Brasil do período de 2012 a 2019.
Como a tabela SIH-RD tratada e filtrada que temos é do período de 2018 até 2019 e apenas do Estado de São Paulo, iremos utilizar o mesmo subconjunto.

In [8]:
%time
sql_t_nasc_vivos = return_sql_query("SELECT CODMUNRES, ano_nasc as Ano, count(*) as Qtd_Total  \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2019 AND \
                    IDADEMAE>=20 AND \
                    IDADEMAE<=34 \
                    GROUP BY CODMUNRES, ano_nasc")
resp_sql_t_nasc_vivos = requests.post(os.path.join(url_base,'sql_query'), json=sql_t_nasc_vivos)
record_columns=resp_sql_t_nasc_vivos.json()['columns']
record_rows=resp_sql_t_nasc_vivos.json()['rows']

fetch_all_data_from_response(resp_sql_t_nasc_vivos, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_t_nasc_vivos = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs
.............................................(y)
Record rows length : 44668


In [9]:
%time
# fonte do dado populacional
# http://tabnet.datasus.gov.br/cgi/deftohtm.exe?popsvs/cnv/popbr.def
df_pop_fem_20_34 = pd.read_csv('./tabelas_tabnet/mulheres_20_34_por_municipio_limpa.csv', ';')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [10]:
df_pop_fem_20_34.head()

,Municipio,2012,2013,2014,2015,2016,2017,2018,2019
0,110001,3083,3072,3058,3036,2985,2928,2866,2801
1,110037,1612,1615,1615,1611,1613,1612,1609,1603
2,110040,2193,2266,2338,2406,2488,2568,2651,2714
3,110034,2065,2030,1993,1950,1898,1842,1777,1723
4,110002,13363,13566,13754,13941,14110,14271,14449,14574


In [11]:
df_t_nasc_vivos.head()

,CODMUNRES,Ano,Qtd_Total
0,110000,2012,1
1,110000,2013,6
2,110000,2014,2
3,110000,2015,4
4,110000,2016,1


In [12]:
years = []
for col in df_pop_fem_20_34:
    if col != 'Municipio':
        years.append(col)

all_entries = []
for index, row in df_pop_fem_20_34.iterrows():
    row_array = row.values.tolist()
    for i in range(len(row_array)):
        if i != 0:
            pop = row_array[i]
            if pop == '-':
                pop = row_array[i+1]
            all_entries.append([row_array[0], years[i-1], int(pop)])

            
df_pop_fem_20_34_by_codmunres = pd.DataFrame(all_entries, columns=['CODMUNRES', 'Ano', 'Pop'])

df_pop_fem_20_34_by_codmunres.head()

,CODMUNRES,Ano,Pop
0,110001,2012,3083
1,110001,2013,3072
2,110001,2014,3058
3,110001,2015,3036
4,110001,2016,2985


In [13]:
df_t_nasc_vivos['Ano'] = df_t_nasc_vivos['Ano'].astype(int)
df_pop_fem_20_34_by_codmunres['CODMUNRES'] = df_pop_fem_20_34_by_codmunres['CODMUNRES'].astype(str)
df_pop_fem_20_34_by_codmunres['Ano'] = df_pop_fem_20_34_by_codmunres['Ano'].astype(int)
df_pop_fem_20_34_by_codmunres['Pop'] = df_pop_fem_20_34_by_codmunres['Pop'].astype(int)
df_merge = pd.merge(df_t_nasc_vivos, df_pop_fem_20_34_by_codmunres, how='right', on=['CODMUNRES','Ano'])
df_merge = df_merge.fillna(0)
df_merge['Ratio'] = df_merge['Qtd_Total']/df_merge['Pop']
df_merge.head()

,CODMUNRES,Ano,Qtd_Total,Pop,Ratio
0,110001,2012,287.0,3083,0.093091
1,110001,2013,277.0,3072,0.090169
2,110001,2014,236.0,3058,0.077175
3,110001,2015,277.0,3036,0.091238
4,110001,2016,244.0,2985,0.081742


In [14]:
%time
df_merge.to_csv('./csv_files/QR-29-taxa_fecundidade_20_34.csv',index=False)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.78 µs
